# Data Structures Implementation

### Record

In [6]:
# from Record import *
import os


class Block:
    def __init__(self, disk_name):
        """ Constructor """
        # open disk file
        if (os.path.isfile('./' + disk_name)):
            self.disk = open(disk_name, "r+")
        else:
            self.disk = open(disk_name, "w+")
        # set size of block
        self.max_size = 8
        # set number of operations (read/write)
        self.n_op = 0
        # set list of records
        self.records = []
        # record size
        self.record_size = 138
        self.pos = 0

    def __del__(self):
        """ Destructor """
        # persist last records to disk
        self.persist(self.disk.tell())
        # close disk file
        self.disk.close()

    def persist(self, pos):
        """ Persists current block's records to the disk file """
        # add 1 to number of operations
        self.n_op += 1
        # position the pointer at the writing location
        self.disk.seek(pos)
        # persist each block record to disk file
        for rec in self.records:
            self.disk.write(str(rec))
        self.disk.flush()
        # reset list of records
        self.records = []

    def write(self, pos, rec):
        """ Write record rec in position pos of the disk file """
        if (len(self.records) >= self.max_size):
            self.persist(pos)
        self.records += [rec]
        self.pos = pos

    def read(self, pos):
        """ Add records of pos position in disk file to this """
        self.n_op += 1
        self.records = []
        self.disk.seek(pos)
        while (len(self.records) < self.max_size):
            self.records += [self.disk.read(self.record_size)]


# r = Record("11111111111;54.037.661-5;estermoro@gmail.com;06/01/1952;Feminino;Yuri Matheus Antonia;5942.00")
# b = Block("out.txt")
# b.write(500, str(r))
# b.persist(500)
# b.read(0)

### Record

In [10]:
class Record:
    def __init__(self, string):
        """ Split argument (record) by ; character and set member variables """
        # set splited record
        field = string.split(';')
        # set cpf
        self.cpf = field[0]
        # set rg
        self.rg = field[1]
        # set email
        self.email = field[2] + '\0' * \
            (40 - len(field[2])) if len(field[2]) < 40 else field[2][:40]
        # set data de nascimento
        self.nasc = field[3]
        # set sexo
        self.sexo = field[4] + '\0' * \
            (9 - len(field[4])) if len(field[4]) < 9 else field[4][:9]
        # set nome
        self.nome = field[5] + '\0'*(40 - len(field[5])
                                     ) if len(field[5]) < 40 else field[5][:40]
        # set salario
        self.salario = field[6] + '\0' * \
            (10 - len(field[6])) if len(field[6]) < 10 else field[6][:10]

    def __str__(self):
        """ Return all member variables concatenated """
        return self.cpf+";"+self.rg+";"+self.email+";"+self.nasc+";"+self.sexo+";"+self.nome+";"+self.salario

### B-Tree

In [8]:
#As seen in https://gist.github.com/natekupp/1763661
class BTreeNode(object):
    """A B-Tree Node.
    
    attributes
    =====================
    leaf : boolean, determines whether this node is a leaf.
    keys : list, a list of keys internal to this node
    c : list, a list of children of this node
    """
    def __init__(self, leaf=False):
        self.leaf = leaf
        self.keys = []
        self.c    = []
        self.pos=0 #added node's content
        
    def __str__(self):
        if self.leaf:
            return "Leaf BTreeNode with {0} keys\n\tK:{1}\n\tC:{2}\n".format(len(self.keys), self.keys, self.c)
        else:
            return "Internal BTreeNode with {0} keys, {1} children\n\tK:{2}\n\n".format(len(self.keys), len(self.c), self.keys, self.c)


class BTree(object):
    def __init__(self, t):
        self.root = BTreeNode(leaf=True)
        self.t    = t
    
    def search(self, k, x=None):
        """Search the B-Tree for the key k.
        
        args
        =====================
        k : Key to search for
        x : (optional) Node at which to begin search. Can be None, in which case the entire tree is searched.
        
        """
        if isinstance(x, BTreeNode):
            i = 0
            while i < len(x.keys) and k > x.keys[i]:    # look for index of k
                i += 1
            if i < len(x.keys) and k == x.keys[i]:       # found exact match
                return (x, i)
            elif x.leaf:                                # no match in keys, and is leaf ==> no match exists
                return None
            else:                                       # search children
                return self.search(k, x.c[i])
        else:                                           # no node provided, search root of tree
            return self.search(k, self.root)
        
    def insert(self, k):
        r = self.root
        if len(r.keys) == (2*self.t) - 1:     # keys are full, so we must split
            s         = BTreeNode()
            self.root = s
            s.c.insert(0, r)                  # former root is now 0th child of new root s
            self._split_child(s, 0)            
            self._insert_nonfull(s, k)
        else:
            self._insert_nonfull(r, k)
    
    def _insert_nonfull(self, x, k):
        i = len(x.keys) - 1
        if x.leaf:
            # insert a key
            x.keys.append(0)
            while i >= 0 and k < x.keys[i]:
                x.keys[i+1] = x.keys[i]
                i -= 1
            x.keys[i+1] = k
        else:
            # insert a child
            while i >= 0 and k < x.keys[i]:
                i -= 1
            i += 1
            if len(x.c[i].keys) == (2*self.t) - 1:
                self._split_child(x, i)
                if k > x.keys[i]:
                    i += 1
            self._insert_nonfull(x.c[i], k)
        
    def _split_child(self, x, i):
        t = self.t
        y = x.c[i]
        z = BTreeNode(leaf=y.leaf)
        
        # slide all children of x to the right and insert z at i+1.
        x.c.insert(i+1, z)
        x.keys.insert(i, y.keys[t-1])
        
        # keys of z are t to 2t - 1,
        # y is then 0 to t-2
        z.keys = y.keys[t:(2*t - 1)]
        y.keys = y.keys[0:(t-1)]
        
        # children of z are t to 2t els of y.c
        if not y.leaf:
            z.c = y.c[t:(2*t)]
            y.c = y.c[0:(t-1)]    
        
    def __str__(self):
        r = self.root
        return r.__str__() + '\n'.join([child.__str__() for child in r.c])  

### Heap

In [15]:
from bplustree import BPlusTree, StrSerializer #pip install btreeplus
#see https://github.com/NicolasLM/bplustree
from os import getcwd
from sys import byteorder


class Heap:
    def __init__(self, disk_name="Heap.cbd", indexBy=[], indexBTree=True, indexBPlusTree=False):
        maxDegreeBTree = 10
        self.r_block = Block(disk_name)
        self.w_block = Block(disk_name)
        self.indexes = {}
        self.indexBTree = indexBTree
        self.indexBPlusTree = indexBPlusTree
        for i in indexBy:
            if indexBPlusTree:
                self.indexes.update({i:BPlusTree(getcwd()+"\\"+i+".index",serializer=StrSerializer(),key_size=128)})
            else:
                if indexBTree:
                    self.indexes.update({i:BTree(maxDegreeBTree)})
                else:
                    self.indexes.update({i:{}})

    def __del__(self):
        if self.indexBPlusTree:
            for i in indexBy:
                self.indexes[i].close()

    def insert(self, string):
        rec = Record(string)
        for i in self.indexes:
            if self.indexBPlusTree:
                self.indexes[i].insert(getattr(rec,i),self.w_block.disk.tell().to_bytes(4,byteorder),True) #True for replacing node, if already existant
            else:
                if self.indexBTree:
                    self.indexes[i].insert(getattr(rec,i))
                    self.indexes[i].search(getattr(rec,i))[0].pos=self.w_block.disk.tell()
                else:
                    self.indexes[i].update({getattr(rec,i):self.w_block.disk.tell()})
        self.w_block.write(self.w_block.disk.tell(), rec)

    def join(self, other_heap, field, other_field=""):
        if not other_field:
            other_field=field
        pos, other_pos = 0, 0
        self.r_block.read(pos)
        while(self.r_block.records[0]):
            for i in self.r_block.records:
                if not i:
                    break
                if other_field in other_heap.indexes:  # if field is indexed
                    if other_heap.indexBPlusTree:
                        other_heap.r_block.read(int.from_bytes(other_heap.indexes[other_field].get(getattr(Record(i),field),byteorder)))
                    else:
                        if other_heap.indexBTree:
                            other_heap.r_block.read(other_heap.indexes[other_field].search(getattr(Record(i),field))[0].pos)
                        else:
                            other_heap.r_block.read(other_heap.indexes[other_field][getattr(Record(i), field)]*other_heap.r_block.record_size)
                    for j in other_heap.r_block.records:
                        if not j:
                            break
                        if getattr(Record(i), field) == getattr(Record(j), other_field):
                            print(i+"\n"+j+"\n")
                else:  # if field is NOT indexed
                    other_pos = 0
                    other_heap.r_block.read(other_pos)
                    while(other_heap.r_block.records[0]):
                        for j in other_heap.r_block.records:
                            if not j:
                                break
                            if getattr(Record(i), field) == getattr(Record(j), other_field):
                                print(i+"\n"+j+"\n")
                        other_pos += other_heap.r_block.max_size*other_heap.r_block.record_size
                        other_heap.r_block.read(other_pos)
            pos += self.r_block.max_size*self.r_block.record_size
            self.r_block.read(pos)


#a=Heap("teste1.cbd",indexBy=["nome"])
#a.insert("11111111111;54.037.661-5;estermoro@gmail.com;06/01/1952;Feminino;Yuri Matheus Antonia;5942.00")
#a.insert("33333333333;54.037.661-5;estermoro@gmail.com;06/01/1952;Feminino;Yuri Matheus Antonia;5942.00")
#b=Heap("teste2.cbd",indexBy=["nome"])
#b.insert("22222222222;54.037.661-5;estermoro@gmail.com;06/01/1952;Feminino;Yuri Matheus Antonia;5942.00")

#a.w_block.persist(0) #debug-only
#b.w_block.persist(0) #debug-only

#a.join(b, "nome")

#del a, b

### Sorted

In [17]:
class Sorted:
    def __init__(self, disk_name="Sorted.cbd", indexBy=[]):
        self.disk_name = disk_name
        self.r_block = Block(disk_name)
        self.w_block = Block(disk_name)
        self.indexes = {}
        for i in indexBy:
            self.indexes.update({i: open(i+"_"+disk_name, "w+")})

    def __del__(self):
        for i in self.indexes:
            self.indexes[i].close()

    def insert(self, string):
        rec = Record(string)
        self.w_block.write(self.w_block.disk.tell(), rec)
        self.w_block.persist(self.w_block.disk.tell())
        allRecords = []
        pos = 0
        self.r_block.read(pos)
        while self.r_block.records[0]:
            allRecords += self.r_block.records
            pos += self.r_block.max_size*self.r_block.record_size
            self.r_block.read(pos)
        allRecords.sort()
        sortedFile = open(self.disk_name, "w")
        sortedFile.write("".join(allRecords))
        sortedFile.flush()
        # + indexing implementation

    def join(self, other_sorted, field, other_field=""):
        if not other_field:
            other_field=field
        pos, other_pos, pos_inside_block, other_pos_inside_block = 0, 0, 0, 0 #*_inside_block marks a record inside a block
        self.r_block.read(pos)
        while(self.r_block.records[0]):
            other_sorted.r_block.read(other_pos)
            while(other_sorted.r_block.records[0]):
                if not self.r_block.records[pos_inside_block]:
                    pos_inside_block=0
                    pos+=self.r_block.max_size*self.r_block.record_size
                    self.r_block.read(pos)
                    break
                if not other_sorted.r_block.records[other_pos_inside_block]:
                    other_pos_inside_block=0
                    other_pos+=other_sorted.r_block.max_size*other_sorted.r_block.record_size
                    other_sorted.r_block.read(pos)
                    break
                if(getattr(Record(self.r_block.records[pos_inside_block]),field)==getattr(Record(other_sorted.r_block.records[other_pos_inside_block]),other_field)):
                    print(self.r_block.records[pos_inside_block]+"\n"+other_sorted.r_block.records[other_pos_inside_block]+"\n")
                    pos_inside_block+=1 #pair found, moves left table's pointer
                else:
                    if(getattr(Record(self.r_block.records[pos_inside_block]),field)<getattr(Record(other_sorted.r_block.records[other_pos_inside_block]),other_field)):
                          pos_inside_block+=1 #moves left table's pointer
                    else:
                          other_pos_inside_block+=1 #moves right table's pointer
                if(pos_inside_block>=self.r_block.max_size): #loads left table's next block
                    pos_inside_block=0
                    pos+=self.r_block.max_size*self.r_block.record_size
                    self.r_block.read(pos)
                if(other_pos_inside_block>=other_sorted.r_block.max_size): #loads right table's next block
                    other_pos_inside_block=0
                    other_pos+=other_sorted.r_block.max_size*other_sorted.r_block.record_size
                    other_sorted.r_block.read(other_pos)
                

# a = Sorted()
# a.insert("11111111111;54.037.661-5;estermoro@gmail.com;06/01/1952;Feminino;Yuri Matheus Antonia;5942.00")
# a.insert("33331111111;54.037.661-5;estermoro@gmail.com;06/01/1952;Feminino;Yuri Matheus Antonia;5942.00")
# a.insert("22222222222;54.037.661-5;estermoro@gmail.com;06/01/1952;Feminino;Yuri Matheus Antonia;5942.00")

#a=Sorted("teste1.cbd")
#a.insert("11111111111;54.037.661-5;estermoro@gmail.com;06/01/1952;Feminino;Yuri Matheus Antonia;5942.00")
#a.insert("33333333333;54.037.661-5;estermoro@gmail.com;06/01/1952;Feminino;Yuri Matheus Antonia;5942.00")
#b=Sorted("teste2.cbd")
#b.insert("22222222222;54.037.661-5;estermoro@gmail.com;06/01/1952;Feminino;Yuri Matheus Antonia;5942.00")

#a.join(b, "nome")

### Hash

In [18]:
# from Record import *
# from Block import *
# from BTree import *
from bplustree import BPlusTree, StrSerializer #pip install btreeplus
#see https://github.com/NicolasLM/bplustree
from os import getcwd
from sys import byteorder

class Hash:
    def __init__(self, disk_name="Hash.cbd", indexBy=[], indexBTree=True, indexBPlusTree=False):
        maxDegreeBTree = 10
        self.r_block = Block(disk_name)
        self.w_block = Block(disk_name)
        self.indexes={}
        self.indexBTree = indexBTree
        self.indexBPlusTree = indexBPlusTree
        self.hashedPositions = []
        for i in indexBy:
            if indexBPlusTree:
                self.indexes.update({i:BPlusTree(getcwd()+"\\"+i+".index",serializer=StrSerializer(),key_size=128)})
            else:
                if indexBTree:
                    self.indexes.update({i:BTree(maxDegreeBTree)})
                else:
                    self.indexes.update({i:{}})

    def __del__(self):
        if self.indexBPlusTree:
            for i in indexBy:
                self.indexes[i].close()

    def insert(self, string):
        rec = Record(string)
        self.w_block.write(abs(hash(rec.cpf)//10**6)*self.w_block.max_size*self.w_block.record_size, rec)
        self.w_block.persist(abs(hash(rec.cpf)//10**6)*self.w_block.max_size*self.w_block.record_size)
        self.hashedPositions += [abs(hash(rec.cpf)//10**6)*self.w_block.max_size*self.w_block.record_size]
        for i in self.indexes:
            if self.indexBPlusTree:
                self.indexes[i].insert(getattr(rec,i),abs(hash(rec.cpf)//10**6)*self.w_block.max_size*self.w_block.record_size.to_bytes(4,byteorder),True) #True for replacing node, if already existant
            else:
                if self.indexBTree:
                    self.indexes[i].insert(getattr(rec,i))
                    self.indexes[i].search(getattr(rec,i))[0].pos=abs(hash(rec.cpf)//10**6)*self.w_block.max_size*self.w_block.record_size
                else:
                    self.indexes[i].update({getattr(rec,i):abs(hash(rec.cpf)//10**6)*self.w_block.max_size*self.w_block.record_size})

    def join(self,other_hash,field,other_field=""):
        if not other_field:
            other_field=field
        for pos in self.hashedPositions:
            self.r_block.read(pos)
            for i in self.r_block.records:
                if i=='\x00'*self.r_block.record_size or not i:
                    break
                if field=="cpf": #if field is primary key cpf
                    other_hash.r_block.read(abs(hash(Record(i).cpf)//10**6)*self.w_block.max_size*self.w_block.record_size)
                    if other_hash.r_block.records[0]!='\x00'*self.r_block.record_size:
                        print(i+"\n"+str(other_hash.r_block.records[0])+"\n")
                else:
                    if other_field in other_hash.indexes: #if field is indexed
                        if other_hash.indexBPlusTree:
                            other_hash.r_block.read(int.from_bytes(other_hash.indexes[other_field].get(getattr(Record(i),field),byteorder)))
                        else:
                            if other_hash.indexBTree:
                                other_hash.r_block.read(other_hash.indexes[other_field].search(getattr(Record(i),field))[0].pos)
                            else:
                                other_hash.r_block.read(other_hash.indexes[other_field][getattr(Record(i),field)])
                        for j in other_hash.r_block.records:
                            if not j:
                                break
                            if getattr(Record(i),field)==getattr(Record(j),other_field):
                                print(i+"\n"+j+"\n")
                    else:
                        other_pos = 0
                        other_hash.r_block.read(other_pos)
                        while(other_hash.r_block.records[0]):
                            for j in other_hash.r_block.records:
                                if j=='\x00'*other_hash.r_block.record_size or not j:
                                    break
                                if getattr(Record(i), field) == getattr(Record(j), other_field):
                                    print(i+"\n"+j+"\n")
                            other_pos += other_hash.r_block.max_size*other_hash.r_block.record_size
                            other_hash.r_block.read(other_pos)

#a=Hash("teste1.cbd",indexBy=["nome"])
#a.insert("11111111111;54.037.661-5;estermoro@gmail.com;06/01/1952;Feminino;Yuri Matheus Antonia;5942.00")
#b=Hash("teste2.cbd",indexBy=["nome"])
#b.insert("11111111111;54.037.661-5;estermoro@gmail.com;06/01/1952;Feminino;Yuri Matheus Antonia;5942.00")
#a.join(b,"cpf")

#del a, b

# Main

In [19]:
import os, random

FileNotFoundError: [Errno 2] No such file or directory: '../data-generation/sample100.csv'